### > > > Example notebook < < <
# Connectome manipulations & visualizations
This is the main script for connectome manipulations, which
 - applies a single or a sequence of manipulations to the connectome, as specified in the config

> **IMPORTANT:** Use supplementary notebooks
> - `model_building.ipynb` for connectome model building
> - `structural_comparator.ipynb` for structural comparison of the raw and manipulated connectomes
> - `topological_comparator.ipynb` for topological database creation and comparison of the raw and manipulated connectomes

> **REQUIREMENTS:**
> - [Connectome Manipulator](https://bbpgitlab.epfl.ch/conn/structural/connectome_manipulator)
> - [Blue Brain SNAP](https://github.com/BlueBrain/snap)
> - [Toposample Utilities](https://github.com/BlueBrain/topological_sampling/tree/master/common/toposample_utilities)

In [1]:
# Initialization

""" Global imports """
import os
import numpy as np
import json

""" Local imports """
from connectome_manipulator.connectome_manipulation import connectome_manipulation
from connectome_manipulator.connectome_comparison import structural_comparator

In [30]:
# Connectome manipulation - Configuration
manip_config = {}

""" Circuit """
# manip_config['circuit_path'] = '/gpfs/bbp.cscs.ch/data/scratch/proj83/home/pokorny/circuit-build-S1_v1_manip'
# manip_config['circuit_config'] = 'sonata/circuit_config_EmptyConnectome.json' # SONATA (.json) format; path rel. to 'circuit_path'
# # manip_config['output_path'] = '/gpfs/bbp.cscs.ch/data/scratch/proj83/home/pokorny/circuit-build-S1_v1_manip__TEST__' # Optional
# manip_config['blue_config_to_update'] = 'CircuitConfig_EmptyConnectome' # Optional; path rel. to 'circuit_path'
# manip_config['workflow_template'] = '../templates/bbp-workflow_RegisterCircuit.cfg' # Optional; to create bbp-workflow config from

manip_config['circuit_path'] = '/gpfs/bbp.cscs.ch/data/scratch/proj83/home/pokorny/circuit-build-S1_v1'
manip_config['circuit_config'] = 'sonata/circuit_config_v2.json' # SONATA (.json) format; path rel. to 'circuit_path'
# OLD # manip_config['output_path'] = '/gpfs/bbp.cscs.ch/data/scratch/proj83/home/pokorny/circuit-build-S1_v1_manip__TEST_OPT__' # Optional
output_path = '/gpfs/bbp.cscs.ch/data/scratch/proj83/home/pokorny/circuit-build-S1_v1_manip__TEST01__'
manip_config['blue_config_to_update'] = 'CircuitConfig' # Optional; path rel. to 'circuit_path'
manip_config['workflow_template'] = '../templates/bbp-workflow_RegisterCircuit.cfg' # Optional; to create bbp-workflow config from

# manip_config['circuit_path'] = '/gpfs/bbp.cscs.ch/data/scratch/proj83/home/pokorny/circuit-build'
# manip_config['circuit_config'] = 'sonata/circuit_config.json' # SONATA (.json) format; path rel. to 'circuit_path'
# manip_config['output_path'] = '/gpfs/bbp.cscs.ch/data/scratch/proj83/home/pokorny/circuit-build_manip' # Optional
# manip_config['blue_config_to_update'] = 'CircuitConfig' # Optional; path rel. to 'circuit_path'
# manip_config['workflow_template'] = '../templates/bbp-workflow_RegisterCircuit.cfg' # Optional; to create bbp-workflow config from

# # O1v5-SONATA
# manip_config['circuit_path'] = '/gpfs/bbp.cscs.ch/data/scratch/proj83/home/pokorny/O1v5-SONATA'
# manip_config['circuit_config'] = 'sonata/circuit_config.json' # SONATA (.json) format; path rel. to 'circuit_path'
# # manip_config['population_name'] = '<name>' # Optional; name of edges population, in case there are more than one
# manip_config['output_path'] = '/gpfs/bbp.cscs.ch/data/scratch/proj83/home/pokorny/O1v5-SONATA_manip__TEST__' # Optional
# manip_config['blue_config_to_update'] = 'CircuitConfig_TC_BlobStim' # Optional; path rel. to 'circuit_path'
# manip_config['workflow_template'] = '../templates/bbp-workflow_RegisterCircuit.cfg' # Optional; to create bbp-workflow config from

# SSCx-HexO1
# manip_config['circuit_path'] = '/gpfs/bbp.cscs.ch/project/proj83/home/pokorny/SimplifiedConnectomeModels/circuits/SSCx-Bio_M-20200805-HexO1'
# manip_config['circuit_config'] = 'sonata/circuit_config.json' # SONATA (.json) format; path rel. to 'circuit_path'
# # manip_config['output_path'] = #<to be set below>
# manip_config['blue_config_to_update'] = 'CircuitConfig_TC' # Optional; path rel. to 'circuit_path'
# manip_config['workflow_template'] = '../templates/bbp-workflow_RegisterCircuit.cfg' # Optional; to create bbp-workflow config from

# TESTING: SSCx projections
# manip_config['circuit_path'] = '/gpfs/bbp.cscs.ch/project/proj83/home/pokorny/SSCx/projections'
# manip_config['circuit_config'] = 'VPM_config.json' # SONATA (.json) format; path rel. to 'circuit_path'

""" General settings """
manip_config['seed'] = 3210
manip_config['N_split_nodes'] = 1

""" Manipulation (name + sequence of manipulation functions with arguments) """
# manip_config['manip'] = {'name': 'NoManip', 'fcts': [{'source': 'no_manipulation', 'kwargs': {}}]} # Just for testing
# manip_config['manip'] = {'name': 'NoManipV2', 'fcts': [{'source': 'no_manipulation', 'kwargs': {}}]} # Just for testing

# manip_config['manip'] = {'name': 'RemoveAllSynToL5', 'fcts': [{'source': 'syn_removal', 'kwargs': {'sel_src': 'All', 'sel_dest': 'Layer5', 'amount_pct': 100.0}}]}
# manip_config['manip'] = {'name': 'RemoveAllConnToL5', 'fcts': [{'source': 'conn_removal', 'kwargs': {'sel_src': 'All', 'sel_dest': 'Layer5', 'amount_pct': 100.0}}]}

# manip_config['manip'] = {'name': 'Remove90pctAllSyn', 'fcts': [{'source': 'syn_removal', 'kwargs': {'amount_pct': 90.0}}]}
# manip_config['manip'] = {'name': 'Remove90pctAllConn', 'fcts': [{'source': 'conn_removal', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'amount_pct': 90.0}}]}
# manip_config['manip'] = {'name': 'RemoveConnEEMask100pct', 'fcts': [{'source': 'conn_removal', 'kwargs': {'sel_src': 'Excitatory', 'sel_dest': 'Excitatory', 'amount_pct': 100.0, 'conn_mask_file': 'conn_mask_EE_TEST.npz'}}]}

# manip_config['manip'] = {'name': 'RemoveAllConnWith1To10Syn', 'fcts': [{'source': 'conn_removal', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'amount_pct': 100.0, 'min_syn_per_conn': 1, 'max_syn_per_conn': 10}}]}
# manip_config['manip'] = {'name': 'RemoveAllConnWith5To10Syn', 'fcts': [{'source': 'conn_removal', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'amount_pct': 100.0, 'min_syn_per_conn': 5, 'max_syn_per_conn': 10}}]}
# manip_config['manip'] = {'name': 'RemoveAllConnWithOver20Syn', 'fcts': [{'source': 'conn_removal', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'amount_pct': 100.0, 'min_syn_per_conn': 21}}]}
# manip_config['manip'] = {'name': 'RemoveAllConnExcept5To10Syn', 'fcts': [{'source': 'conn_removal', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'amount_pct': 100.0, 'max_syn_per_conn': 4}},
#                                                                          {'source': 'conn_removal', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'amount_pct': 100.0, 'min_syn_per_conn': 11}}]}

# manip_config['manip'] = {'name': 'Remove50pctAllSyn', 'fcts': [{'source': 'syn_removal', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'amount_pct': 50.0}}]}
# manip_config['manip'] = {'name': 'Remove50pctAllSynKeepGsyn', 'fcts': [{'source': 'syn_removal', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'amount_pct': 50.0, 'rescale_gsyn': True}}]}
# manip_config['manip'] = {'name': 'Remove50pctAllSynKeepConn', 'fcts': [{'source': 'syn_removal', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'amount_pct': 50.0, 'keep_conns': True}}]}
# manip_config['manip'] = {'name': 'Remove50pctAllSynKeepConnKeepGsyn', 'fcts': [{'source': 'syn_removal', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'amount_pct': 50.0, 'keep_conns': True, 'rescale_gsyn': True}}]}
# manip_config['manip'] = {'name': 'RemoveAllSynKeepConn', 'fcts': [{'source': 'syn_removal', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'amount_pct': 100.0, 'keep_conns': True}}]}
# manip_config['manip'] = {'name': 'RemoveAllSynKeepConnKeepGsyn', 'fcts': [{'source': 'syn_removal', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'amount_pct': 100.0, 'keep_conns': True, 'rescale_gsyn': True}}]}
# manip_config['manip'] = {'name': 'ReduceToPointNeuronModel', 'fcts': [{'source': 'syn_removal', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'amount_pct': 100.0, 'keep_conns': True, 'rescale_gsyn': True}},
#                                                                       {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'afferent_section_id', 'new_value': {'mode': 'setval', 'value': 0}, 'amount_pct': 100.0}},
#  *** TODO: ***                                                        {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'afferent_section_pos', 'new_value': {'mode': 'setval', 'value': 0.0}, 'amount_pct': 100.0}},
#  *   -Reset afferent x/y/z properties, since no longer consistent *   {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'afferent_section_type', 'new_value': {'mode': 'setval', 'value': 0}, 'amount_pct': 100.0}},
#  ***                                                                  {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'afferent_segment_id', 'new_value': {'mode': 'setval', 'value': 0}, 'amount_pct': 100.0}},
#                                                                       {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'afferent_segment_offset', 'new_value': {'mode': 'setval', 'value': 0.0}, 'amount_pct': 100.0}}]}

# manip_config['manip'] = {'name': 'SubsampleSyn10pct', 'fcts': [{'source': 'syn_subsampling', 'kwargs': {'keep_pct': 10.0}}]}

# manip_config['manip'] = {'name': 'RewireAxonsL4PC-L5PC', 'fcts': [{'source': 'axon_rewiring', 'kwargs': {'sel_grp1': {'mtype': ['L4_TPC', 'L4_UPC']}, 'sel_grp2': {'mtype': ['L5_TPC:A', 'L5_TPC:B', 'L5_TPC:C', 'L5_UPC']}, 'R': 100.0, 'amount_pct': 100.0}}]}
# manip_config['manip'] = {'name': 'ShuffleAxonsL4TPC', 'fcts': [{'source': 'axon_shuffling', 'kwargs': {'sel_grp': {'mtype': 'L4_TPC'}, 'R': 100.0, 'amount_pct': 100.0}}]}
# manip_config['manip'] = {'name': 'ShuffleAxonsL456EXC', 'fcts': [{'source': 'axon_shuffling', 'kwargs': {'sel_grp': {'layer': [4, 5, 6], 'synapse_class': 'EXC'}, 'R': 100.0, 'amount_pct': 100.0}}]}

# manip_config['manip'] = {'name': 'ScaleNrrpX2', 'fcts': [{'source': 'syn_prop_alteration', 'kwargs': {'syn_prop': 'n_rrp_vesicles', 'new_value': {'mode': 'scale', 'factor': 2}, 'amount_pct': 100.0}}]}
# manip_config['manip'] = {'name': 'ScaleGsynX2', 'fcts': [{'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'conductance', 'new_value': {'mode': 'scale', 'factor': 2}, 'amount_pct': 100.0}}]}
# manip_config['manip'] = {'name': 'ScaleGsynX0.5', 'fcts': [{'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'conductance', 'new_value': {'mode': 'scale', 'factor': 0.5}, 'amount_pct': 100.0}}]}
# manip_config['manip'] = {'name': 'ScaleDelayX2', 'fcts': [{'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'delay', 'new_value': {'mode': 'scale', 'factor': 2}, 'amount_pct': 100.0}}]}
# manip_config['manip'] = {'name': 'ScaleDelayX0.5', 'fcts': [{'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'delay', 'new_value': {'mode': 'scale', 'factor': 0.5}, 'amount_pct': 100.0}}]}

# manip_config['manip'] = {'name': 'ScaleGsyn0p1L4aff', 'fcts': [{'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'Layer4', 'syn_prop': 'conductance', 'new_value': {'mode': 'scale', 'factor': 0.1}, 'amount_pct': 100.0}}]}
# manip_config['manip'] = {'name': 'ScaleGsyn0p1L4eff', 'fcts': [{'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'Layer4', 'sel_dest': 'All', 'syn_prop': 'conductance', 'new_value': {'mode': 'scale', 'factor': 0.1}, 'amount_pct': 100.0}}]}
# manip_config['manip'] = {'name': 'ScaleGsyn0p1ExcV1', 'fcts': [{'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_filter': {'syn_type_id': list(range(100, 200))}, 'syn_prop': 'conductance', 'new_value': {'mode': 'scale', 'factor': 0.1}, 'amount_pct': 100.0}}]}
# manip_config['manip'] = {'name': 'ScaleGsyn0p1ExcV2', 'fcts': [{'source': 'syn_prop_alteration', 'kwargs': {'sel_src': {'synapse_class': 'EXC'}, 'sel_dest': 'All', 'syn_prop': 'conductance', 'new_value': {'mode': 'scale', 'factor': 0.1}, 'amount_pct': 100.0}}]}
# manip_config['manip'] = {'name': 'ScaleGsyn2p0L4Exc-L4Exc', 'fcts': [{'source': 'syn_prop_alteration', 'kwargs': {'sel_src': {'layer': 4, 'synapse_class': 'EXC'}, 'sel_dest': {'layer': 4, 'synapse_class': 'EXC'}, 'syn_prop': 'conductance', 'new_value': {'mode': 'scale', 'factor': 2.0}, 'amount_pct': 100.0}}]}

# manip_config['manip'] = {'name': 'ResetPreSynPos', 'fcts': [{'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'efferent_center_x', 'new_value': {'mode': 'setval', 'value': 0.0}, 'amount_pct': 100.0}},
#                                                             {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'efferent_center_y', 'new_value': {'mode': 'setval', 'value': 0.0}, 'amount_pct': 100.0}},
#                                                             {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'efferent_center_z', 'new_value': {'mode': 'setval', 'value': 0.0}, 'amount_pct': 100.0}},
#                                                             {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'efferent_surface_x', 'new_value': {'mode': 'setval', 'value': 0.0}, 'amount_pct': 100.0}},
#                                                             {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'efferent_surface_y', 'new_value': {'mode': 'setval', 'value': 0.0}, 'amount_pct': 100.0}},
#                                                             {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'efferent_surface_z', 'new_value': {'mode': 'setval', 'value': 0.0}, 'amount_pct': 100.0}}]}

# manip_config['manip'] = {'name': 'ResetPreSynSect', 'fcts': [{'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'efferent_section_id', 'new_value': {'mode': 'setval', 'value': 0}, 'amount_pct': 100.0}},
#                                                              {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'efferent_section_pos', 'new_value': {'mode': 'setval', 'value': 0.0}, 'amount_pct': 100.0}},
#                                                              {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'efferent_section_type', 'new_value': {'mode': 'setval', 'value': 0}, 'amount_pct': 100.0}}]}

# manip_config['manip'] = {'name': 'ResetPreSynSegm', 'fcts': [{'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'efferent_segment_id', 'new_value': {'mode': 'setval', 'value': 0}, 'amount_pct': 100.0}},
#                                                              {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'efferent_segment_offset', 'new_value': {'mode': 'setval', 'value': 0.0}, 'amount_pct': 100.0}}]}

# manip_config['manip'] = {'name': 'ResetPostSynPos', 'fcts': [{'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'afferent_center_x', 'new_value': {'mode': 'setval', 'value': 0.0}, 'amount_pct': 100.0}},
#                                                              {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'afferent_center_y', 'new_value': {'mode': 'setval', 'value': 0.0}, 'amount_pct': 100.0}},
#                                                              {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'afferent_center_z', 'new_value': {'mode': 'setval', 'value': 0.0}, 'amount_pct': 100.0}},
#                                                              {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'afferent_surface_x', 'new_value': {'mode': 'setval', 'value': 0.0}, 'amount_pct': 100.0}},
#                                                              {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'afferent_surface_y', 'new_value': {'mode': 'setval', 'value': 0.0}, 'amount_pct': 100.0}},
#                                                              {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'afferent_surface_z', 'new_value': {'mode': 'setval', 'value': 0.0}, 'amount_pct': 100.0}}]}

# manip_config['manip'] = {'name': 'ResetPostSynSect', 'fcts': [{'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'afferent_section_id', 'new_value': {'mode': 'setval', 'value': 0}, 'amount_pct': 100.0}},
#                                                               {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'afferent_section_pos', 'new_value': {'mode': 'setval', 'value': 0.0}, 'amount_pct': 100.0}},
#                                                               {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'afferent_section_type', 'new_value': {'mode': 'setval', 'value': 0}, 'amount_pct': 100.0}}]}

# manip_config['manip'] = {'name': 'ResetPostSynSegm', 'fcts': [{'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'afferent_segment_id', 'new_value': {'mode': 'setval', 'value': 0}, 'amount_pct': 100.0}},
#                                                               {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'afferent_segment_offset', 'new_value': {'mode': 'setval', 'value': 0.0}, 'amount_pct': 100.0}}]}

# manip_config['manip'] = {'name': 'ResetPostSynSectId', 'fcts': [{'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'afferent_section_id', 'new_value': {'mode': 'setval', 'value': 0}, 'amount_pct': 100.0}}]}
# manip_config['manip'] = {'name': 'ResetPostSynSectPos', 'fcts': [{'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'afferent_section_pos', 'new_value': {'mode': 'setval', 'value': 0.0}, 'amount_pct': 100.0}}]}
# manip_config['manip'] = {'name': 'ResetPostSynSectType', 'fcts': [{'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'afferent_section_type', 'new_value': {'mode': 'setval', 'value': 0}, 'amount_pct': 100.0}}]}

# manip_config['manip'] = {'name': 'InvalPreSynAll', 'fcts': [{'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'efferent_center_x', 'new_value': {'mode': 'setval', 'value': np.nan}, 'amount_pct': 100.0}},
#                                                             {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'efferent_center_y', 'new_value': {'mode': 'setval', 'value': np.nan}, 'amount_pct': 100.0}},
#                                                             {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'efferent_center_z', 'new_value': {'mode': 'setval', 'value': np.nan}, 'amount_pct': 100.0}},
#                                                             {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'efferent_surface_x', 'new_value': {'mode': 'setval', 'value': np.nan}, 'amount_pct': 100.0}},
#                                                             {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'efferent_surface_y', 'new_value': {'mode': 'setval', 'value': np.nan}, 'amount_pct': 100.0}},
#                                                             {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'efferent_surface_z', 'new_value': {'mode': 'setval', 'value': np.nan}, 'amount_pct': 100.0}},
#                                                             {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'efferent_section_id', 'new_value': {'mode': 'setval', 'value': -1}, 'amount_pct': 100.0}},
#                                                             {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'efferent_section_pos', 'new_value': {'mode': 'setval', 'value': np.nan}, 'amount_pct': 100.0}},
#                                                             {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'efferent_section_type', 'new_value': {'mode': 'setval', 'value': -1}, 'amount_pct': 100.0}},
#                                                             {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'efferent_segment_id', 'new_value': {'mode': 'setval', 'value': -1}, 'amount_pct': 100.0}},
#                                                             {'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'efferent_segment_offset', 'new_value': {'mode': 'setval', 'value': np.nan}, 'amount_pct': 100.0}}]}

# manip_config['manip'] = {'name': 'RndCond', 'fcts': [{'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'conductance', 'new_value': {'mode': 'randval', 'rng': 'lognormal', 'kwargs': {'mean': 0.5, 'sigma': 0.5}, 'range': [0.0, 5.0]}, 'amount_pct': 100.0}}]}
# manip_config['manip'] = {'name': 'InvalCondScale', 'fcts': [{'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'conductance_scale_factor', 'new_value': {'mode': 'setval', 'value': 0.0}, 'amount_pct': 100.0}}]}
# manip_config['manip'] = {'name': 'InvalDecayTime', 'fcts': [{'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'decay_time', 'new_value': {'mode': 'setval', 'value': 0.0}, 'amount_pct': 100.0}}]}
# manip_config['manip'] = {'name': 'InvalDeprTime', 'fcts': [{'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'depression_time', 'new_value': {'mode': 'setval', 'value': 0.0}, 'amount_pct': 100.0}}]}
# manip_config['manip'] = {'name': 'InvalFacilTime', 'fcts': [{'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'facilitation_time', 'new_value': {'mode': 'setval', 'value': 0.0}, 'amount_pct': 100.0}}]}
# manip_config['manip'] = {'name': 'InvalSpineLen', 'fcts': [{'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'spine_length', 'new_value': {'mode': 'setval', 'value': 0.0}, 'amount_pct': 100.0}}]}
# manip_config['manip'] = {'name': 'InvalSynType', 'fcts': [{'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'syn_type_id', 'new_value': {'mode': 'setval', 'value': -1}, 'amount_pct': 100.0}}]}
# manip_config['manip'] = {'name': 'InvalUHill', 'fcts': [{'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'u_hill_coefficient', 'new_value': {'mode': 'setval', 'value': 0.0}, 'amount_pct': 100.0}}]}
# manip_config['manip'] = {'name': 'InvalUSyn', 'fcts': [{'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_prop': 'u_syn', 'new_value': {'mode': 'setval', 'value': 0.0}, 'amount_pct': 100.0}}]}

# manip_config['manip'] = {'name': 'ShuffleExc', 'fcts': [{'source': 'axon_shuffling', 'kwargs': {'sel_grp': 'Excitatory', 'R': np.inf, 'amount_pct': 100.0}}]}

# manip_config['manip'] = {'name': 'ConnRewireExcOrd1', 'fcts': [{'source': 'conn_rewiring', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_class': 'EXC', 'amount_pct': 100.0, 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb1stOrder.pickle', 'delay_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/DistDepDelay.pickle'}}]}
# manip_config['manip'] = {'name': 'ConnRewireOrd1KeepIndeg', 'fcts': [{'source': 'conn_rewiring', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_class': 'EXC', 'keep_indegree': True, 'amount_pct': 100.0, 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb1stOrder.pickle', 'delay_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/DistDepDelay.pickle'}},
#                                                                      {'source': 'conn_rewiring', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_class': 'INH', 'keep_indegree': True, 'amount_pct': 100.0, 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb1stOrder.pickle', 'delay_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/DistDepDelay.pickle'}}]}
# manip_config['manip'] = {'name': 'ConnRewireOrd1', 'fcts': [{'source': 'conn_rewiring', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_class': 'EXC', 'keep_indegree': False, 'gen_method': 'duplicate_sample', 'amount_pct': 100.0, 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb1stOrder.pickle', 'delay_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/DistDepDelay.pickle'}},
#                                                             {'source': 'conn_rewiring', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_class': 'INH', 'keep_indegree': False, 'gen_method': 'duplicate_sample', 'amount_pct': 100.0, 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb1stOrder.pickle', 'delay_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/DistDepDelay.pickle'}}]}
# manip_config['manip'] = {'name': 'ConnRewireOrd2', 'fcts': [{'source': 'conn_rewiring', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_class': 'EXC', 'amount_pct': 100.0, 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb2ndOrder.pickle', 'delay_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/DistDepDelay.pickle'}},
#                                                             {'source': 'conn_rewiring', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_class': 'INH', 'amount_pct': 100.0, 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb2ndOrder.pickle', 'delay_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/DistDepDelay.pickle'}}]}
# manip_config['manip'] = {'name': f'ConnRewireEEOrd1_SEED{manip_config["seed"]}', 'fcts': [{'source': 'conn_rewiring', 'kwargs': {'estimation_run': True, 'opt_nconn': False, 'p_scale': 1.0, 'sel_src': 'Excitatory', 'sel_dest': 'Excitatory', 'syn_class': 'EXC', 'amount_pct': 100.0, 'keep_indegree':False, 'gen_method': 'duplicate_sample', 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb1stOrderEE.json', 'delay_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/DistDepDelay.json'}}]}
# manip_config['manip'] = {'name': f'ConnRewireEEOrd2_SEED{manip_config["seed"]}', 'fcts': [{'source': 'conn_rewiring', 'kwargs': {'estimation_run': False, 'opt_nconn': True, 'p_scale': 1.2415070750696142 * 0.9993138716848566 * 0.9998443950169793, 'sel_src': 'Excitatory', 'sel_dest': 'Excitatory', 'syn_class': 'EXC', 'amount_pct': 100.0, 'keep_indegree':False, 'gen_method': 'duplicate_sample', 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb2ndOrderEE.json', 'delay_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/DistDepDelay.json'}}]}
# manip_config['manip'] = {'name': f'ConnRewireEEOrd4R_SEED{manip_config["seed"]}', 'fcts': [{'source': 'conn_rewiring', 'kwargs': {'sel_src': 'Excitatory', 'sel_dest': 'Excitatory', 'syn_class': 'EXC', 'amount_pct': 100.0, 'keep_indegree': False, 'gen_method': 'duplicate_sample', 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb4thOrderEEFlatspace.json', 'delay_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/DistDepDelay.json', 'pos_map_file': '../working_dir/model_building/circuit-build-S1_v1/model/FlatPosMapping.json'}}]}
# manip_config['manip'] = {'name': f'ConnRewireEEOrd4R_SEED{manip_config["seed"]}40umV3', 'fcts': [{'source': 'conn_rewiring', 'kwargs': {'sel_src': 'Excitatory', 'sel_dest': 'Excitatory', 'syn_class': 'EXC', 'amount_pct': 100.0, 'keep_indegree':False, 'gen_method': 'duplicate_sample', 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb4thOrderEEFlatspace40umV3.json', 'delay_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/DistDepDelay.json', 'pos_map_file': '../working_dir/model_building/circuit-build-S1_v1/model/FlatPosMapping.json'}}]}

# manip_config['manip'] = {'name': 'Add100pctSyn', 'fcts': [{'source': 'syn_addition', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'amount': {'type': 'pct', 'value': 100.0}, 'method': 'duplicate', 'rescale_gsyn': False}}]}
# manip_config['manip'] = {'name': 'Add100pctSynKeepGsyn', 'fcts': [{'source': 'syn_addition', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'amount': {'type': 'pct', 'value': 100.0}, 'method': 'duplicate', 'rescale_gsyn': True}}]}
# manip_config['manip'] = {'name': 'Add50pctInhSyn', 'fcts': [{'source': 'syn_addition', 'kwargs': {'sel_src': 'Inhibitory', 'sel_dest': 'All', 'amount': {'type': 'pct', 'value': 50.0}, 'method': 'duplicate', 'rescale_gsyn': False}}]}
# manip_config['manip'] = {'name': 'Add100pctSynPerConn', 'fcts': [{'source': 'syn_addition', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'amount': {'type': 'pct_per_conn', 'value': 100.0}, 'method': 'duplicate', 'rescale_gsyn': False}}]}
# manip_config['manip'] = {'name': 'Add100pctSynPerConnKeepGsyn', 'fcts': [{'source': 'syn_addition', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'amount': {'type': 'pct_per_conn', 'value': 100.0}, 'method': 'duplicate', 'rescale_gsyn': True}}]}
# manip_config['manip'] = {'name': 'Min10SynPerConn', 'fcts': [{'source': 'syn_addition', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'amount': {'type': 'minval_per_conn', 'value': 10}, 'method': 'duplicate', 'rescale_gsyn': False}}]}
# manip_config['manip'] = {'name': 'Min10SynPerConnKeepGsyn', 'fcts': [{'source': 'syn_addition', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'amount': {'type': 'minval_per_conn', 'value': 10}, 'method': 'duplicate', 'rescale_gsyn': True}}]}
# manip_config['manip'] = {'name': 'Add5to10SynPerConn', 'fcts': [{'source': 'syn_addition', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'amount': {'type': 'rnd_per_conn', 'value': [5, 10]}, 'method': 'duplicate', 'rescale_gsyn': False}}]}

# manip_config['manip'] = {'name': 'Remove90pctConnExcToL4', 'fcts': [{'source': 'conn_removal', 'kwargs': {'sel_src': 'Excitatory', 'sel_dest': 'Layer4', 'amount_pct': 90.0}}]}
# manip_config['manip'] = {'name': 'RemoveAllSynExcToL4KeepConnKeepGsyn', 'fcts': [{'source': 'syn_removal', 'kwargs': {'sel_src': 'Excitatory', 'sel_dest': 'Layer4', 'amount_pct': 100.0, 'keep_conns': True, 'rescale_gsyn': True}}]}
# manip_config['manip'] = {'name': 'ScaleGsynX5ExcToL4', 'fcts': [{'source': 'syn_prop_alteration', 'kwargs': {'sel_src': 'Excitatory', 'sel_dest': 'Layer4', 'syn_prop': 'conductance', 'new_value': {'mode': 'scale', 'factor': 5}, 'amount_pct': 100.0}}]}
# manip_config['manip'] = {'name': 'Add100pctSynExcToL4', 'fcts': [{'source': 'syn_addition', 'kwargs': {'sel_src': 'Excitatory', 'sel_dest': 'Layer4', 'amount': {'type': 'pct', 'value': 100.0}, 'method': 'duplicate', 'rescale_gsyn': False}}]}
# manip_config['manip'] = {'name': 'Add10pctSynL3ToL4RndModel', 'fcts': [{'source': 'syn_addition', 'kwargs': {'sel_src': 'Layer3', 'sel_dest': 'Layer4', 'amount': {'type': 'pct', 'value': 10.0}, 'method': 'derive', 'rescale_gsyn': False, 'props_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnPropsPerMType.pickle'}}]}
# manip_config['manip'] = {'name': 'Add10pctSynPerConnL3ToL4RndModel', 'fcts': [{'source': 'syn_addition', 'kwargs': {'sel_src': 'Layer3', 'sel_dest': 'Layer4', 'amount': {'type': 'pct_per_conn', 'value': 10.0}, 'method': 'derive', 'rescale_gsyn': False, 'props_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnPropsPerMType.pickle'}}]}
# manip_config['manip'] = {'name': 'Add10pctSynL3ToL4LoadFromStruct', 'fcts': [{'source': 'syn_addition', 'kwargs': {'sel_src': 'Layer3', 'sel_dest': 'Layer4', 'amount': {'type': 'pct', 'value': 10.0}, 'method': 'load', 'rescale_gsyn': False, 'ext_edges_config_file': os.path.join(manip_config['circuit_path'], 'sonata/struct_circuit_config.json')}}]}
# manip_config['manip'] = {'name': 'Add10pctSynPerConnL3ToL4LoadFromStruct', 'fcts': [{'source': 'syn_addition', 'kwargs': {'sel_src': 'Layer3', 'sel_dest': 'Layer4', 'amount': {'type': 'pct_per_conn', 'value': 10.0}, 'method': 'load', 'rescale_gsyn': False, 'ext_edges_config_file': os.path.join(manip_config['circuit_path'], 'sonata/struct_circuit_config.json')}}]}
# manip_config['manip'] = {'name': 'ConnRewireExcToExcOrd1KeepIndeg', 'fcts': [{'source': 'conn_rewiring', 'kwargs': {'sel_src': 'Excitatory', 'sel_dest': 'Excitatory', 'syn_class': 'EXC', 'amount_pct': 100.0, 'keep_indegree': True, 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb1stOrder.pickle', 'delay_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/DistDepDelay.pickle'}}]}
# manip_config['manip'] = {'name': 'ConnRewireExcToExcOrd1', 'fcts': [{'source': 'conn_rewiring', 'kwargs': {'sel_src': 'Excitatory', 'sel_dest': 'Excitatory', 'syn_class': 'EXC', 'amount_pct': 100.0, 'keep_indegree': False, 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb1stOrder.pickle', 'delay_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/DistDepDelay.pickle'}}]}
# manip_config['manip'] = {'name': 'ConnRewireExcToExcOrd1TEST', 'fcts': [{'source': 'conn_rewiring', 'kwargs': {'sel_src': 'Excitatory', 'sel_dest': 'Excitatory', 'syn_class': 'EXC', 'amount_pct': 100.0, 'keep_indegree': False, 'gen_method': 'duplicate_sample', 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb1stOrder.json', 'delay_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/DistDepDelay.json', 'pos_map_file': '../working_dir/model_building/circuit-build-S1_v1/model/FlatPosMapping.json'}}]}
# manip_config['manip'] = {'name': 'ConnRewireExcToExcOrd1KeepIndegTEST', 'fcts': [{'source': 'conn_rewiring', 'kwargs': {'sel_src': 'Excitatory', 'sel_dest': 'Excitatory', 'syn_class': 'EXC', 'amount_pct': 100.0, 'keep_indegree': True, 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb1stOrder.json', 'delay_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/DistDepDelay.json', 'pos_map_file': '../working_dir/model_building/circuit-build-S1_v1/model/FlatPosMapping.json'}}]}
# manip_config['manip'] = {'name': 'ConnRewireOrd1KeepIndeg', 'fcts': [{'source': 'conn_rewiring', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_class': 'EXC', 'keep_indegree': True, 'amount_pct': 100.0, 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb1stOrder.pickle', 'delay_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/DistDepDelay.pickle'}},
#                                                                      {'source': 'conn_rewiring', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_class': 'INH', 'keep_indegree': True, 'amount_pct': 100.0, 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb1stOrder.pickle', 'delay_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/DistDepDelay.pickle'}}]}
# manip_config['manip'] = {'name': 'ConnRewireOrd1KeepIndegNoDelayCorr', 'fcts': [{'source': 'conn_rewiring', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_class': 'EXC', 'keep_indegree': True, 'amount_pct': 100.0, 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb1stOrder.pickle'}},
#                                                                                 {'source': 'conn_rewiring', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_class': 'INH', 'keep_indegree': True, 'amount_pct': 100.0, 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb1stOrder.pickle'}}]}
# manip_config['manip'] = {'name': 'ConnRewireOrd1', 'fcts': [{'source': 'conn_rewiring', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_class': 'EXC', 'keep_indegree': False, 'amount_pct': 100.0, 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb1stOrder.pickle', 'delay_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/DistDepDelay.pickle'}},
#                                                             {'source': 'conn_rewiring', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_class': 'INH', 'keep_indegree': False, 'amount_pct': 100.0, 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb1stOrder.pickle', 'delay_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/DistDepDelay.pickle'}}]}
# manip_config['manip'] = {'name': 'ConnRewireOrd1NoDelayCorr', 'fcts': [{'source': 'conn_rewiring', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_class': 'EXC', 'keep_indegree': False, 'amount_pct': 100.0, 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb1stOrder.pickle'}},
#                                                                        {'source': 'conn_rewiring', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_class': 'INH', 'keep_indegree': False, 'amount_pct': 100.0, 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb1stOrder.pickle'}}]}
# manip_config['manip'] = {'name': 'ConnRewireOrd2', 'fcts': [{'source': 'conn_rewiring', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_class': 'EXC', 'keep_indegree': False, 'amount_pct': 100.0, 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb2ndOrder.pickle', 'delay_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/DistDepDelay.pickle'}},
#                                                             {'source': 'conn_rewiring', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_class': 'INH', 'keep_indegree': False, 'amount_pct': 100.0, 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb2ndOrder.pickle', 'delay_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/DistDepDelay.pickle'}}]}
# manip_config['manip'] = {'name': 'ConnRewireOrd2Flatspace', 'fcts': [{'source': 'conn_rewiring', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_class': 'EXC', 'keep_indegree': False, 'amount_pct': 100.0, 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb2ndOrderFlatspace.pickle', 'delay_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/DistDepDelay.pickle', 'pos_map_file': '../working_dir/model_building/circuit-build-S1_v1/model/FlatPosMapping.pickle'}},
#                                                                      {'source': 'conn_rewiring', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_class': 'INH', 'keep_indegree': False, 'amount_pct': 100.0, 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb2ndOrderFlatspace.pickle', 'delay_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/DistDepDelay.pickle', 'pos_map_file': '../working_dir/model_building/circuit-build-S1_v1/model/FlatPosMapping.pickle'}}]}
# manip_config['manip'] = {'name': 'ConnRewireOrd3', 'fcts': [{'source': 'conn_rewiring', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_class': 'EXC', 'keep_indegree': False, 'amount_pct': 100.0, 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb3rdOrder.pickle', 'delay_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/DistDepDelay.pickle'}},
#                                                             {'source': 'conn_rewiring', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_class': 'INH', 'keep_indegree': False, 'amount_pct': 100.0, 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb3rdOrder.pickle', 'delay_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/DistDepDelay.pickle'}}]}
# manip_config['manip'] = {'name': 'ConnRewireOrd3Flatspace', 'fcts': [{'source': 'conn_rewiring', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_class': 'EXC', 'keep_indegree': False, 'amount_pct': 100.0, 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb3rdOrderFlatspace.pickle', 'delay_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/DistDepDelay.pickle', 'pos_map_file': '../working_dir/model_building/circuit-build-S1_v1/model/FlatPosMapping.pickle'}},
#                                                                      {'source': 'conn_rewiring', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_class': 'INH', 'keep_indegree': False, 'amount_pct': 100.0, 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb3rdOrderFlatspace.pickle', 'delay_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/DistDepDelay.pickle', 'pos_map_file': '../working_dir/model_building/circuit-build-S1_v1/model/FlatPosMapping.pickle'}}]}
# manip_config['manip'] = {'name': 'ConnRewireOrd4', 'fcts': [{'source': 'conn_rewiring', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_class': 'EXC', 'keep_indegree': False, 'amount_pct': 100.0, 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb4thOrder.pickle', 'delay_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/DistDepDelay.pickle'}},
#                                                             {'source': 'conn_rewiring', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_class': 'INH', 'keep_indegree': False, 'amount_pct': 100.0, 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb4thOrder.pickle', 'delay_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/DistDepDelay.pickle'}}]}
# manip_config['manip'] = {'name': 'ConnRewireOrd4Flatspace', 'fcts': [{'source': 'conn_rewiring', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_class': 'EXC', 'keep_indegree': False, 'amount_pct': 100.0, 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb4thOrderFlatspace.pickle', 'delay_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/DistDepDelay.pickle', 'pos_map_file': '../working_dir/model_building/circuit-build-S1_v1/model/FlatPosMapping.pickle'}},
#                                                                      {'source': 'conn_rewiring', 'kwargs': {'sel_src': 'All', 'sel_dest': 'All', 'syn_class': 'INH', 'keep_indegree': False, 'amount_pct': 100.0, 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb4thOrderFlatspace.pickle', 'delay_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/DistDepDelay.pickle', 'pos_map_file': '../working_dir/model_building/circuit-build-S1_v1/model/FlatPosMapping.pickle'}}]}

# manip_config['manip'] = {'name': 'ConnRewireOrd1KeepIndeg', 'fcts': [{'source': 'conn_rewiring', 'kwargs': {'sel_src': {'synapse_class': 'EXC'}, 'sel_dest': {'synapse_class': 'EXC'}, 'syn_class': 'EXC', 'keep_indegree': True, 'amount_pct': 100.0, 'prob_model_file': '../working_dir/model_building/circuit-build/ConnProb1stOrder__synapse_class_pathways/model/ConnProb1stOrder__synapse_class__EXC-EXC.pickle', 'delay_model_file': '../working_dir/model_building/circuit-build/DistDepDelay__synapse_class_pathways/model/DistDepDelay__synapse_class__EXC-EXC.pickle'}}]}
# manip_config['manip'] = {'name': 'ConnRewireOrd1', 'fcts': [{'source': 'conn_rewiring', 'kwargs': {'sel_src': {'synapse_class': 'EXC'}, 'sel_dest': {'synapse_class': 'EXC'}, 'syn_class': 'EXC', 'keep_indegree': False, 'gen_method': 'duplicate_sample', 'amount_pct': 100.0, 'prob_model_file': '../working_dir/model_building/circuit-build/ConnProb1stOrder__synapse_class_pathways/model/ConnProb1stOrder__synapse_class__EXC-EXC.pickle', 'delay_model_file': '../working_dir/model_building/circuit-build/DistDepDelay__synapse_class_pathways/model/DistDepDelay__synapse_class__EXC-EXC.pickle'}}]}
# manip_config['manip'] = {'name': 'ConnRewireL4EEOrd1', 'fcts': [{'source': 'conn_rewiring', 'kwargs': {'sel_src': 'Layer4', 'sel_dest': 'Layer4', 'syn_class': 'EXC', 'amount_pct': 100.0, 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb1stOrder.pickle', 'delay_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/DistDepDelay.pickle'}}]}

# manip_config['manip'] = {'name': 'EmptyConnectome', 'fcts': [{'source': 'conn_extraction', 'kwargs': {}}]}
# manip_config['manip'] = {'name': 'ExtractL4', 'fcts': [{'source': 'conn_extraction', 'kwargs': {'target_name': 'Layer4'}}]}

# manip_config['manip'] = {'name': 'ConnRewireEEOrd1TEST', 'fcts': [{'source': 'conn_rewiring', 'kwargs': {'sel_src': {'synapse_class': 'EXC'}, 'sel_dest': {'synapse_class': 'EXC'}, 'syn_class': 'EXC', 'keep_indegree': False, 'gen_method': 'duplicate_sample', 'amount_pct': 10.0, 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb1stOrder.json', 'delay_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/DistDepDelay.json'}}]}
# manip_config['manip'] = {'name': 'ConnRewireEEOrd1TEST', 'fcts': [{'source': 'conn_rewiring', 'kwargs': {'sel_src': {'synapse_class': 'EXC'}, 'sel_dest': {'synapse_class': 'EXC'}, 'syn_class': 'EXC', 'keep_indegree': False, 'reuse_conns': False, 'gen_method': 'duplicate_randomize', 'amount_pct': 100.0, 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb1stOrder.json', 'delay_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/DistDepDelay.json', 'props_model_file': '/gpfs/bbp.cscs.ch/project/proj83/home/pokorny/SimplifiedConnectomeModels/model_building/SSCx-Bio_M-20200805-HexO1/model/ConnPropsPerPathwayEE.json'}}]}
# manip_config['manip'] = {'name': 'ConnRewireEEOrd1TEST2', 'fcts': [{'source': 'conn_rewiring', 'kwargs': {'sel_src': {'synapse_class': 'EXC'}, 'sel_dest': {'synapse_class': 'EXC'}, 'syn_class': 'EXC', 'keep_indegree': True, 'reuse_conns': True, 'amount_pct': 100.0, 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb1stOrder.json', 'delay_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/DistDepDelay.json'}}]}
# manip_config['manip'] = {'name': 'ConnRewireEEOrd1TEST3', 'fcts': [{'source': 'conn_rewiring', 'kwargs': {'sel_src': {'synapse_class': 'EXC'}, 'sel_dest': {'layer': 1}, 'syn_class': 'EXC', 'keep_indegree': False, 'reuse_conns': False, 'gen_method': 'duplicate_randomize', 'amount_pct': 10.0, 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb1stOrder.json', 'delay_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/DistDepDelay.json', 'props_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnPropsPerMType.json'}}]}

# Empty connectome
# manip_config['manip'] = {'name': 'ConnWiringOrd1TEST', 'fcts': [{'source': 'conn_wiring', 'kwargs': {'amount_pct': 10.0, 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb1stOrder.json', 'nsynconn_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/NSynPerConnPerMType.json', 'pos_map_file': '../working_dir/model_building/circuit-build-S1_v1/model/FlatPosMapping.json', 'delay_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/DistDepDelay.json', 'morph_ext': 'h5'}}]}
# manip_config['manip'] = {'name': 'ConnWiringOrd2TEST2', 'fcts': [{'source': 'conn_wiring', 'kwargs': {'amount_pct': 10.0, 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb2ndOrderFlatspace.json', 'nsynconn_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/NSynPerConnPerMType.json', 'pos_map_file': '../working_dir/model_building/circuit-build-S1_v1/model/FlatPosMapping.json', 'delay_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/DistDepDelay.json', 'morph_ext': 'h5'}}]}

# O1v5-SONATA
# manip_config['manip'] = {'name': 'Remove50pctAllSyn', 'fcts': [{'source': 'syn_removal', 'kwargs': {'sel_src': 'Mosaic', 'sel_dest': 'Mosaic', 'amount_pct': 50.0}}]}

# SSCx-HexO1
# manip_config['manip'] = {'name': 'Remove50pctAllSyn', 'fcts': [{'source': 'syn_removal', 'kwargs': {'sel_src': 'hex_O1', 'sel_dest': 'hex_O1', 'amount_pct': 50.0}}]}
# manip_config['output_path'] = manip_config['circuit_path'] + '__' + manip_config['manip']['name']

# NEW COMMAND LINE VERSION (v0.0.5.dev1 or later)
manip_config["manip"] = {"name": "ConnRewireEEOrd1TEST", "fcts": [{"source": "conn_rewiring", "kwargs": {"sel_src": {"synapse_class": "EXC"}, "sel_dest": {"synapse_class": "EXC"}, "syn_class": "EXC", "keep_indegree": False, "gen_method": "duplicate_sample", "amount_pct": 10.0, "prob_model_spec": {"file": "/gpfs/bbp.cscs.ch/home/pokorny/JupyterLab/git/connectome_manipulator/examples/working_dir/model_building/circuit-build-S1_v1/model/ConnProb1stOrder.json"}, "delay_model_spec": {"file": "/gpfs/bbp.cscs.ch/home/pokorny/JupyterLab/git/connectome_manipulator/examples/working_dir/model_building/circuit-build-S1_v1/model/DistDepDelay.json"}}}]}

# Connectome manipulation - Apply manipulation
# OLD # connectome_manipulation.main(manip_config, do_profiling=True, do_resume=False, keep_parquet=True)

tmp_config_file = "manip_config_tmp.json"
with open(tmp_config_file, "w") as f:
    json.dump(manip_config, f, indent=2)

# Run command (executed in current folder)
run_cmd = f"connectome-manipulator manipulate-connectome {tmp_config_file} --output-dir {output_path} --profile --convert-to-sonata --splits=1"
print("LAUNCH COMMAND:")
print(f"cd {os.getcwd()}")
print("module purge")
print("module load unstable")
print("module load parquet-converters/0.8.0")
print(". /gpfs/bbp.cscs.ch/home/pokorny/ReWiringKernel/bin/activate")
print("connectome-manipulator --version")
print(run_cmd)

LAUNCH COMMAND:
module purge
module load unstable
module load parquet-converters/0.8.0
. /gpfs/bbp.cscs.ch/home/pokorny/ReWiringKernel/bin/activate
connectome-manipulator --version
connectome-manipulator manipulate-connectome manip_config_tmp.json --output-dir /gpfs/bbp.cscs.ch/data/scratch/proj83/home/pokorny/circuit-build-S1_v1_manip__TEST01__ --profile --convert-to-sonata --splits=1


In [ ]:
# [WARNING] Data logging not possible!

In [22]:
# Estimate global probability scaling factor for matching #connections (from log file of estimation or actual rewiring run)
log_type = 'Estimation' # 'Rewiring'...actual rewiring, 'Estimation'...estimation run
N_split = 1
data_logs = [f'/gpfs/bbp.cscs.ch/data/scratch/proj83/home/pokorny/circuit-build-S1_v1_manip__TEST_OPT__/logs/connectome_manipulation.20230126T122037.{log_type}Stats_{i + 1}_{N_split}.npz' for i in range(N_split)]

conn_count_orig_all = 0 # Overall input connection count
conn_count_pred_all = 0 # Overall output connection count after rewiring (average prediction)
conn_count_actu_all = 0 # Overall output connection count after rewiring (actual value)
conn_count_orig_sel = 0 # Input connection count within selected rewiring target
conn_count_pred_sel = 0 # Output connection count within selected rewiring target after rewiring (average prediction)
conn_count_actu_sel = 0 # Output connection count within selected rewiring target after rewiring (actual value)
diff_from_avg = []
for dlog in data_logs:
    stats_dict = np.load(dlog)
    conn_count_orig_sel += np.sum(stats_dict['input_conn_count_sel'])
    conn_count_pred_sel += np.sum(stats_dict['output_conn_count_sel_avg'])
    conn_count_orig_all += np.sum(stats_dict['input_conn_count'])
    conn_count_pred_all += np.sum(stats_dict['output_conn_count_sel_avg']) + np.sum(stats_dict['input_conn_count']) - np.sum(stats_dict['input_conn_count_sel'])
    if log_type == 'Rewiring':
        conn_count_actu_sel += np.sum(stats_dict['output_conn_count_sel'])
        assert stats_dict['output_conn_count'] == np.sum(stats_dict['output_conn_count_sel']) + np.sum(stats_dict['input_conn_count']) - np.sum(stats_dict['input_conn_count_sel'])
        conn_count_actu_all += stats_dict['output_conn_count']
        diff_from_avg += list(stats_dict['output_conn_count_sel'] - stats_dict['output_conn_count_sel_avg'])

if log_type == 'Rewiring':
    print('<ACTUAL REWIRED INSTANCE>')
    print(f'Overall: #Conns_orig = {conn_count_orig_all}, #Conns_rewired = {conn_count_actu_all}, DIFF = {conn_count_actu_all - conn_count_orig_all} ({100.0 * (conn_count_actu_all - conn_count_orig_all) / conn_count_orig_all:.2f}%)')
    print(f'Within wiring target: #Conns_orig = {conn_count_orig_sel}, #Conns_rewired = {conn_count_actu_sel}, DIFF = {conn_count_actu_sel - conn_count_orig_sel} ({100.0 * (conn_count_actu_sel - conn_count_orig_sel) / conn_count_orig_sel:.2f}%)')
    print(f'Deviation from average prediction of ingoing #conns of {len(diff_from_avg)} post-neurons: MIN = {np.min(diff_from_avg)}, MAX = {np.max(diff_from_avg)}, #MISMATCH = {np.sum(np.array(diff_from_avg) != 0)}, OVERALL #Conn DIFF = {np.sum(diff_from_avg)}')
    print()

p_scale = conn_count_orig_sel / conn_count_pred_sel
print('<AVERAGE PREDICTION>')
print(f'Overall: #Conns_orig = {conn_count_orig_all}, #Conns_predicted = {conn_count_pred_all}, DIFF = {conn_count_pred_all - conn_count_orig_all} ({100.0 * (conn_count_pred_all - conn_count_orig_all) / conn_count_orig_all:.2f}%)')
print(f'Within wiring target: #Conns_orig = {conn_count_orig_sel}, #Conns_predicted = {conn_count_pred_sel}, DIFF = {conn_count_pred_sel - conn_count_orig_sel} ({100.0 * (conn_count_pred_sel - conn_count_orig_sel) / conn_count_orig_sel:.2f}%)')
print(f'Probability scaling p_scale = {conn_count_orig_sel} / {conn_count_pred_sel} = {p_scale}')

<AVERAGE PREDICTION>
Overall: #Conns_orig = 123636, #Conns_predicted = 124127, DIFF = 491 (0.40%)
Within wiring target: #Conns_orig = 109234, #Conns_predicted = 109725, DIFF = 491 (0.45%)
Probability scaling p_scale = 109234 / 109725 = 0.9955251765778081


In [2]:
# Circuit wiring w/o an existing connectome - Configuration
manip_config = {}

""" Circuit """
# No connectome (for wiring only!)
manip_config['circuit_path'] = '/gpfs/bbp.cscs.ch/data/scratch/proj83/home/pokorny/circuit-build-S1_v1_NoConn'
manip_config['circuit_config'] = 'sonata/circuit_config.json' # SONATA (.json) format; path rel. to 'circuit_path'
manip_config['output_path'] = '/gpfs/bbp.cscs.ch/data/scratch/proj83/home/pokorny/circuit-build-S1_v1_NoConn_Wired__TEST__' # Optional

""" General settings """
manip_config['seed'] = 3210
manip_config['N_split_nodes'] = 1

""" Manipulation (name + sequence of manipulation functions with arguments) """
manip_config['manip'] = {'name': 'ConnWiringOrd1TEST', 'fcts': [{'source': 'conn_wiring', 'kwargs': {'amount_pct': 10.0, 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb1stOrder.json', 'nsynconn_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/NSynPerConnPerMType.json', 'pos_map_file': '../working_dir/model_building/circuit-build-S1_v1/model/FlatPosMapping.json', 'delay_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/DistDepDelay.json', 'morph_ext': 'h5'}}]}

# Circuit wiring w/o an existing connectome - Apply manipulation
connectome_manipulation.main_wiring(manip_config, do_profiling=True, do_resume=False, keep_parquet=True, overwrite_edges=True)


PROFILING: 
************************************
* RESOURCE PROFILING [initial]     *
************************************
* Max. memory usage (GB):    0.110 *
************************************

INFO: Loading circuit from /gpfs/bbp.cscs.ch/data/scratch/proj83/home/pokorny/circuit-build-S1_v1_NoConn/sonata/circuit_config.json (N_split=1)
INFO: No edges population defined between nodes "All" and "All"
INFO: Split 1/1: Wiring connectome targeting 5924 neurons
INFO: APPLYING MANIPULATION "ConnWiringOrd1TEST"
INFO: >>Step 1 of 1: source=conn_wiring, kwargs={'amount_pct': 10.0, 'prob_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/ConnProb1stOrder.json', 'nsynconn_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/NSynPerConnPerMType.json', 'pos_map_file': '../working_dir/model_building/circuit-build-S1_v1/model/FlatPosMapping.json', 'delay_model_file': '../working_dir/model_building/circuit-build-S1_v1/model/DistDepDelay.json', 'morph_ext': 'h5'}


In [3]:
# Create bbp-workflow config for registering original circuit (from template)
connectome_manipulation.create_workflow_config(manip_config['circuit_path'],
                                               os.path.join(manip_config['circuit_path'], manip_config['blue_config_to_update']),
                                               None,
                                               manip_config['circuit_path'],
                                               '../templates/bbp-workflow_RegisterCircuit.cfg')


INFO: Creating file /gpfs/bbp.cscs.ch/data/scratch/proj83/home/pokorny/circuit-build-S1_v1/workflows/bbp-workflow_RegisterCircuit.cfg


In [4]:
# Topological comparison of connectomes - Configuration
# => Prepares topological_comparator and writes config to file
# => Config file to be used in topological_comparator.ipynb which requires BluePy v2 kernel!

assert 'blue_config_to_update' in manip_config.keys(), 'ERROR: BlueConfig required for topological comparator!'

topocomp_config = {}

""" General settings """
circuit_name = os.path.split(manip_config['circuit_path'])[-1]
topocomp_config['working_dir'] = os.path.join('..', 'working_dir', 'topological_comparator', circuit_name)
topocomp_config['out_dir'] = os.path.join('figures', circuit_name)

""" Circuits to compare """
topocomp_config['circuits'] = {'0': {'circuit_config': os.path.join(manip_config['circuit_path'], manip_config['blue_config_to_update']),
                                     'circuit_name': 'Raw'},
                               '1': {'circuit_config': os.path.join(manip_config.get('output_path', manip_config['circuit_path']), manip_config['blue_config_to_update'] + '_' + manip_config['manip']['name']),
                                     'circuit_name': manip_config['manip']['name']}}

# Write config to file
topocomp_config_file = os.path.abspath(os.path.join(topocomp_config['working_dir'], 'topocomp_config__' + '_vs_'.join([v['circuit_name'] for k, v in topocomp_config['circuits'].items()]) + '.json'))
if not os.path.exists(topocomp_config['working_dir']):
    os.makedirs(topocomp_config['working_dir'])
with open(topocomp_config_file, 'w') as f:
    json.dump(topocomp_config, f, indent=2)
print(f"topocomp_config_file = '{topocomp_config_file}'")

topocomp_config_file = '/gpfs/bbp.cscs.ch/home/pokorny/JupyterLab/git/connectome_manipulator/examples/working_dir/topological_comparator/circuit-build-S1_v1/topocomp_config__Raw_vs_NoManip.json'


In [ ]:
assert False, 'STOP HERE'

In [25]:
### LOAD RAW EDGES TABLE (TESTING & DEVELOPMENT)
from bluepysnap.circuit import Circuit
from bluepysnap.sonata_constants import Node
from bluepysnap.sonata_constants import Edge

circuit_path = manip_config['circuit_path'] #'/gpfs/bbp.cscs.ch/data/scratch/proj83/home/pokorny/circuit-build-S1_v1'
circuit_name = os.path.split(circuit_path)[-1]
sonata_config =  os.path.join(circuit_path, 'sonata', 'circuit_config_v2.json') # [Using default name]
c = Circuit(sonata_config)
edges = c.edges['default']
nodes = [edges.source, edges.target]
node_ids = [n.ids() for n in nodes]
edges_table = edges.afferent_edges(node_ids[1], properties=sorted(edges.property_names))

circuit = c
# conns, syn_conn_idx, num_syn_per_conn = np.unique(edges_table[['@source_node', '@target_node']], axis=0, return_inverse=True, return_counts=True)

In [34]:
# CREATE (SPARSE) ADJACENCY MATRIX FROM EDGES TABLE
import scipy.sparse as sps
from connectome_manipulator.access_functions import get_node_ids

# Select sub-matrix (optional)
sel_src = {'node_set': 'Excitatory'}
sel_dest = {'node_set': 'Excitatory'}
sel_ids_src = get_node_ids(nodes[0], sel_src)
sel_ids_dest = get_node_ids(nodes[1], sel_dest)
syn_sel_idx = np.logical_and(np.isin(edges_table['@source_node'], sel_ids_src), np.isin(edges_table['@target_node'], sel_ids_dest))

# # OR Full matrix
# syn_sel_idx = np.ones(edges_table.shape[0]).astype(bool)
# sel_ids_src = node_ids[0]
# sel_ids_dest = node_ids[1]

src_conv = sps.csr_matrix((np.arange(len(sel_ids_src), dtype=int), (np.zeros(len(sel_ids_src), dtype=int), sel_ids_src)))
dest_conv = sps.csr_matrix((np.arange(len(sel_ids_dest), dtype=int), (np.zeros(len(sel_ids_dest), dtype=int), sel_ids_dest)))

conns, syn_conn_idx, num_syn_per_conn = np.unique(edges_table[syn_sel_idx][['@source_node', '@target_node']], axis=0, return_inverse=True, return_counts=True)
conns_reindex = np.array([src_conv[0, conns[:, 0]].toarray().flatten(), dest_conv[0, conns[:, 1]].toarray().flatten()]).T
adj_matrix = sps.csc_matrix((np.full(conns_reindex.shape[0], True), conns_reindex.T.tolist()), shape=(len(sel_ids_src), len(sel_ids_dest)))
# sps.save_npz('adj_matrix_TEST.npz', adj_matrix)

In [ ]:
# ACCESS MORPHOLOGIES (TESTING & DEVELOPMENT)
import neurom as nm
from bluepysnap.morph import MorphHelper

morph_dir = nodes[1].config['morphologies_dir']
tgt_morph = MorphHelper(morph_dir, nodes[1], {'h5v1': os.path.join(morph_dir, 'h5v1'), 'neurolucida-asc': os.path.join(morph_dir, 'ascii')})
get_tgt_morph = lambda node_id: tgt_morph.get(node_id, transform=True, extension='h5') # Access function (incl. transformation!), using .h5 format

## Example: How to compute afferent_center_x/y/z positions of a synapse from afferent_section_id/pos
sec_types = ['Soma', 'Axon', 'Basal', 'Apical']
nrn_id = 0 # Target neuron ID
nrn_morph = get_tgt_morph(nrn_id)
syn = edges_table[edges_table['@target_node'] == nrn_id]
assert syn.size > 0, f'ERROR: No synapses on target neuron {nrn_id}!'
syn = syn.iloc[0]
sec_id = int(syn['afferent_section_id']) - 1 # IMPORTANT: Section IDs in NeuroM morphology don't include soma, so they need to be shifted by -1
if sec_id >=0:
    sec_off = syn['afferent_section_pos']
    print(f'From morphology:  {nm.morphmath.path_fraction_point(nrn_morph.section(sec_id).points, sec_off)} ({nrn_morph.section(sec_id).type})')
else:
    print(f'From morphology:  {nrn_morph.soma.center.astype(float)} (Soma section!)')
print(f"From edges table: {syn[['afferent_center_x', 'afferent_center_y', 'afferent_center_z']].to_numpy()} ({sec_types[int(syn['afferent_section_type'])]})")


In [20]:
### LOAD MANIPULATED EDGES TABLE (TESTING & DEVELOPMENT)
from bluepysnap.circuit import Circuit
from bluepysnap.sonata_constants import Node
from bluepysnap.sonata_constants import Edge

circuit_path_manip = manip_config.get('output_path', manip_config['circuit_path'])
# circuit_name_manip = f'{os.path.split(circuit_path_manip)[-1]}__{manip_config["manip"]["name"]}'
# sonata_config_manip = os.path.join(circuit_path_manip, manip_config['circuit_config'])
sonata_config_manip = os.path.join(circuit_path_manip, os.path.splitext(manip_config['circuit_config'])[0] + f'_{manip_config["manip"]["name"]}' + os.path.splitext(manip_config['circuit_config'])[1])
circuit_manip = Circuit(sonata_config_manip)
edges_manip = circuit_manip.edges['default']
nodes_manip = [edges_manip.source, edges_manip.target]
node_ids_manip = [n.ids() for n in nodes_manip]
edges_table_manip = edges_manip.afferent_edges(node_ids_manip[1], properties=sorted(edges_manip.property_names))


In [42]:
# CREATE (SPARSE) ADJACENCY MATRIX FROM MANIPULATED EDGES TABLE
import scipy.sparse as sps
from connectome_manipulator.access_functions import get_node_ids

# Select sub-matrix (optional)
sel_src = {'node_set': 'Excitatory'}
sel_dest = {'node_set': 'Excitatory'}
sel_ids_src = get_node_ids(nodes_manip[0], sel_src)
sel_ids_dest = get_node_ids(nodes_manip[1], sel_dest)
syn_sel_idx = np.logical_and(np.isin(edges_table_manip['@source_node'], sel_ids_src), np.isin(edges_table_manip['@target_node'], sel_ids_dest))

# # OR Full matrix
# syn_sel_idx = np.ones(edges_table_manip.shape[0]).astype(bool)
# sel_ids_src = node_ids_manip[0]
# sel_ids_dest = node_ids_manip[1]

src_conv = sps.csr_matrix((np.arange(len(sel_ids_src), dtype=int), (np.zeros(len(sel_ids_src), dtype=int), sel_ids_src)))
dest_conv = sps.csr_matrix((np.arange(len(sel_ids_dest), dtype=int), (np.zeros(len(sel_ids_dest), dtype=int), sel_ids_dest)))

conns, syn_conn_idx, num_syn_per_conn = np.unique(edges_table_manip[syn_sel_idx][['@source_node', '@target_node']], axis=0, return_inverse=True, return_counts=True)
conns_reindex = np.array([src_conv[0, conns[:, 0]].toarray().flatten(), dest_conv[0, conns[:, 1]].toarray().flatten()]).T
adj_matrix_manip = sps.csc_matrix((np.full(conns_reindex.shape[0], True), conns_reindex.T.tolist()), shape=(len(sel_ids_src), len(sel_ids_dest)))
# sps.save_npz('adj_matrix_manip_TEST.npz', adj_matrix_manip)

In [3]:
### LOAD EDGES TABLE USED FOR TEST CASES (TESTING & DEVELOPMENT)
from bluepysnap.circuit import Circuit
from bluepysnap.sonata_constants import Node
from bluepysnap.sonata_constants import Edge

circuit_path = '../../tests/data'
sonata_config =  os.path.join(circuit_path, 'circuit_sonata.json')
c = Circuit(sonata_config)
edges = c.edges[c.edges.population_names[0]]
nodes = [edges.source, edges.target]
node_ids = [n.ids() for n in nodes]
edges_table = edges.afferent_edges(node_ids[1], properties=sorted(edges.property_names))

circuit = c
# conns, syn_conn_idx, num_syn_per_conn = np.unique(edges_table[['@source_node', '@target_node']], axis=0, return_inverse=True, return_counts=True)

In [157]:
### RENAME POPULATION IN EXISTING SONATA EDGES (H5) FILE => IN-PLACE, OVERWRITING EXISTING FILE!!
import h5py
h5_edges_file = '/gpfs/bbp.cscs.ch/data/scratch/proj83/home/pokorny/circuit-build-S1_v1/sonata/networks/edges/functional/All/edges_RENAMING_TEST.h5'
with h5py.File(h5_edges_file, 'r+') as h5_file:
    h5_file['edges'].move('default', 'edges_population_abc') # move(source, dest)

# Check results:
# h5_file = h5py.File(h5_edges_file, 'r')
# h5_file['edges'].keys()
# ...
# h5_file.close()

In [3]:
# CHECK DUPLICATES WITHIN NEW CONNECTIONS (from data log)
dd = np.load('/gpfs/bbp.cscs.ch/data/scratch/proj83/home/pokorny/circuit-build-S1_v1/logs/connectome_manipulation.20220805T121736.RewiringIndices_1_1.npz')
out_conns_new = dd['out_conns'][dd['out_syn_conn_idx'][dd['out_syn_new_idx']], :]
conns = dd['out_conns'][np.unique(dd['out_syn_conn_idx'][dd['out_syn_new_idx']]), :]

tgt = []
for c in conns:
    dupl = dd['syn_new_dupl_idx'][np.logical_and(out_conns_new[:, 0] == c[0], out_conns_new[:, 1] == c[1])]
    if len(dupl) != len(np.unique(dupl)):
        # print(f'WARNING: Duplicates found in connection {c}')
        tgt.append(c[1])
print(f'WARNING: Duplicates in connections targetting {np.unique(tgt)}')